In [ ]:
import cv2
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from matplotlib import pyplot as plt
import skimage
import os
import sys
import sqlite3
import shutil
from time import time
sys.path.append(os.environ['REPO_DIR'])
from lib.utils import configuration, run
from matplotlib.path import Path
from shapely.geometry import Polygon

In [2]:
def CDF(x):
    x=np.sort(x)
    size=x.shape[0]
    y=np.arange(0,size)/size
    return x,y


def setup_download_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(s3_fp, local_fp))
    else:
        run('aws s3 cp {0} {1}'.format(s3_fp, local_fp))

def setup_upload_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(local_fp, s3_fp))
    else:
        run('aws s3 cp {0} {1}'.format(local_fp, s3_fp))

In [3]:
stack = 'MD594'
fp = os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt')
setup_download_from_s3(fp, recursive=False)
with open(os.environ['ROOT_DIR']+fp, 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    section_to_filename = {int(idx): fn for fn, idx in fn_idx_tuples}


fname = os.path.join('CSHL_data_processed', stack, 'Annotation.npy')
setup_download_from_s3(fname, recursive=False)
annotation = np.load(os.environ['ROOT_DIR']+fname, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
contours = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})
contours_grouped = contours.groupby('section')
valid_sections = np.sort(contours['section'].unique())

fn = 'CSHL_data_processed/MD589/ThresholdsV2.pkl'
setup_download_from_s3(fn, recursive=False)
thresholds = pickle.load(open(os.environ['ROOT_DIR']+fn,'rb'))

ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE


In [4]:
savepath = 'CSHL_cells_mark/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+stack+'/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+'color/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)

resol = 0.46

paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']

all_structures = paired_structures + singular_structures

margin = 500/0.46

In [5]:
len(valid_sections)

265

In [6]:
%%time
raw_images_root = 'CSHL_data_processed/'+stack+'/'+stack+'_prep2_lossless_gray/'
for section in valid_sections:
    img_fn = raw_images_root + section_to_filename[section] + '_prep2_lossless_gray.tif'
#     setup_download_from_s3(img_fn, recursive=False)
    img = cv2.imread(os.environ['ROOT_DIR']+img_fn, 2)
#     whole = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#     polygons = [(contour['name'], contour['vertices']) \
#             for contour_id, contour in contours_grouped.get_group(section).iterrows()]
#     count = 0    
#     cs = []
#     for contour_id, contour in polygons:
#         structure = contour_id
#         if structure not in all_structures:
#             continue
#         polygon = contour.copy()
#         cs.append(polygon.astype(np.int32))
#         count += 1
#         print(section, structure, count, '/', len(polygons))
#     com = cv2.polylines(whole.copy(), cs, True, [255, 0, 0], 3, lineType=50)
    filename = savepath + str(section) + '.tif'
#     com = cv2.cvtColor(com, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.environ['ROOT_DIR']+filename, img)    

CPU times: user 14min 26s, sys: 3min 21s, total: 17min 47s
Wall time: 20min 44s


In [8]:
img.dtype

dtype('uint8')

In [6]:
%%time
script_dir = os.environ['REPO_DIR']
! python $script_dir/Cell_mark.py 'MD594' 319 

/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_databases/MD594/319.db /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_databases/MD594/319.db
download: s3://mousebrainatlas-data/CSHL_databases/MD594/319.db to ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_databases/MD594/319.db
run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-N70-2015.08.27-01.10.06_MD594_2_0209_prep2_lossless_gray.tif /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-N70-2015.08.27-01.10.06_MD594_2_0209_prep2_lossless_gray.tif
download: s3://

upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/VCA/319.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/VCA/319.png
319 VCA 22
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/VCP/319.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/VCP/319.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/VCP/319.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/VCP/319.png
319 VCP 23
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/AP/319.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/AP/319.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/AP/319.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/AP/319.png
319 AP 24
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/12N/319.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/12N/319.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_

In [3]:
setup_download_from_s3('CSHL_cells_mark/MD594/')

run cmd= aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_cells_mark/MD594/ /nfs/birdstore/U19/Freund_Group/CSHL_cells_mark/MD594/


In [3]:
stack = 'MD594'
fp = os.path.join('/data/CSHL_data_processed', stack, stack + '_sorted_filenames.txt')
with open(fp, 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    section_to_filename = {int(idx): fn for fn, idx in fn_idx_tuples}

fname = os.path.join('CSHL_data_processed', stack, 'Annotation.npy')
s3_fp = 's3://mousebrainatlas-data/' + fname
local_fp = '/data/' + fname
# run('aws s3 cp {0} {1}'.format(s3_fp, local_fp))
annotation = np.load('/data/'+fname, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
contours = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})
contours_grouped = contours.groupby('section')
valid_sections = np.sort(contours['section'].unique())

In [4]:
root_dir = '/data2/Shapeology_Files/'

In [5]:
%%time
savepath = 'Background/'
if not os.path.exists(root_dir+savepath):
    os.mkdir(root_dir+savepath)
savepath = savepath+stack+'_full/'
if not os.path.exists(root_dir+savepath):
    os.mkdir(root_dir+savepath)

raw_images_root = 'CSHL_data_processed/'+stack+'/'+stack+'_prep2_lossless_gray/'
for section in valid_sections: #[:len(valid_sections)//2]:
    img_fn = raw_images_root + section_to_filename[section] + '_prep2_lossless_gray.tif'
#     setup_download_from_s3(img_fn, recursive=False)
    img = cv2.imread(os.environ['ROOT_DIR']+img_fn, 2)
    filename = savepath + str(section) + '.tif'
    cv2.imwrite(root_dir+filename, img)   

CPU times: user 15min 43s, sys: 2min 44s, total: 18min 27s
Wall time: 21min 44s


In [5]:
from glob import glob
source_dir = '/nfs/birdstore/U19/Freund_Group/'
files = [dir for dir in glob(source_dir+'CSHL_cells_mark/MD594/VCA/*')]

In [6]:
%%time
savepath = 'Segmentation/'
if not os.path.exists(root_dir+savepath):
    os.mkdir(root_dir+savepath)
savepath = savepath+stack+'/'
if not os.path.exists(root_dir+savepath):
    os.mkdir(root_dir+savepath)
structure = 'VCP'
savepath = savepath+structure+'/'
if not os.path.exists(root_dir+savepath):
    os.mkdir(root_dir+savepath)

for section in valid_sections[:len(valid_sections)//2]:
    src = glob(source_dir+'CSHL_cells_mark/'+stack+'/'+structure+'/'+ str(section) + '*.png')[0]
    shutil.copy2(src, root_dir+savepath)

CPU times: user 194 ms, sys: 316 ms, total: 510 ms
Wall time: 2.93 s


In [10]:
%%time
savepath = 'Background/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+stack+'/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)

raw_images_root = '/data/CSHL_data_processed/'+stack+'/'+stack+'_prep2_lossless_gray/'
for section in valid_sections:
    img_fn = raw_images_root + section_to_filename[section] + '_prep2_lossless_gray.tif'
#     setup_download_from_s3(img_fn, recursive=False)
    img = cv2.imread(img_fn, 2)
    filename = savepath + str(section) + '.tif'
    cv2.imwrite(os.environ['ROOT_DIR']+filename, img)   

CPU times: user 15min 10s, sys: 2min 55s, total: 18min 5s
Wall time: 22min 44s


In [11]:
%%time
savepath = 'Background/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+stack+'/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)

raw_images_root = '/data/CSHL_data_processed/'+stack+'/'+stack+'_prep2_lossless_gray/'
for section in valid_sections[:len(valid_sections)//2]:
    img_fn = raw_images_root + section_to_filename[section] + '_prep2_lossless_gray.tif'
#     setup_download_from_s3(img_fn, recursive=False)
    img = cv2.imread(img_fn, 2)
    filename = savepath + str(section) + '.tif'
    cv2.imwrite(os.environ['ROOT_DIR']+filename, img)   

CPU times: user 7min 26s, sys: 1min 19s, total: 8min 45s
Wall time: 11min


In [5]:
4*13*13*13*13*12/2

685464.0

In [6]:
13*12*11*10*9/5/4/3/2*4*4*4*4*4

1317888.0

In [7]:
8*7*6*5*6

10080

In [8]:
15*14*13*12*11*10/6/5/4/3/2

5005.0

In [ ]:
TT